<a href="https://colab.research.google.com/github/Devica2000/StanfordTech16LLM/blob/main/CrewAI_Tech16_CW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required pacakges


In [1]:
%pip install --q crewai
%pip install --q -U duckduckgo-search
%pip install --q langchain_google_genai
%pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 306.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 676.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 7.1 M

### Required Packages

In [12]:
import os
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import OpenAI
from crewai import Agent, Task, Crew, Process

In [13]:
from google.colab import userdata
# google_api_key = userdata.get('gemini_key')

### set up the LLM

In [14]:
# # Set gemini pro as llm
# llm = ChatGoogleGenerativeAI(model="gemini-pro",
#                              verbose = True,
#                              temperature = 0.6,
#                              google_api_key=google_api_key)
llm = OpenAI(temperature=0.1, openai_api_key=userdata.get('open_ai_key'))

In [15]:
import os
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')

In [16]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x7bf6614272e0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7bf66217db10>, temperature=0.1, openai_api_key=SecretStr('**********'), openai_proxy='')

### setup tools

In [17]:
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search

from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [18]:
question_to_answer = "Conduct investment analysis on Airbnb"

**Sequential**


In [19]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal=f'Conduct comprehensive research to answer the question {question_to_answer}. With a particular focus on thinking about second order effects.',
  backstory="""You work at a leading analysis firm.
  Your expertise lies in analyzing complex issues.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        search_tool
      ]
)

critic = Agent(
  role='Critic',
  goal='Criticize output of research analysts to ensure the content is really insightful and not generic',
  backstory="""You are a senior analyst with an IQ of 125. You care alot about research and want to ensure insights are actionable""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)

writer = Agent(
  role='Research Writer',
  goal=f'Craft a compelling research report to answer the question: {question_to_answer}',
  backstory="""You are a renowned Research Strategist, known for
  your insightful and engaging research.
  You transform complex concepts into compelling reports. Include a section on potential second order effects.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)


### Tasks to perform

In [20]:
# Create tasks for your agents
task1 = Task(
  description=f"""Conduct a comprehensive research analysis to answer the question: {question_to_answer}""",
  agent=researcher,
  expected_output=f"Conduct a comprehensive research analysis to answer the question: {question_to_answer}"  # Add the expected output field
)

In [21]:
# Create tasks for your agents
task2 = Task(
  description="""Critique the report and insights generated""",
  agent=critic,
  expected_output="A set of criticisms and suggested improvements"  # Add the expected output field
)

In [22]:
task3 = Task(
  description=f"""Using the insights provided, develop a research analysis to answer the question: {question_to_answer}""",
  agent=writer,
  expected_output=f"A full research report of at least 4 paragraphs research analysis answering the question: {question_to_answer}." # Added expected output
)

### Create a Crew

In [24]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, critic, writer],
  tasks=[task1, task2, task3],
  verbose=True, # You can set it to 1 or 2 to different logging levels,
)

In [25]:
crew

Crew(id=e5bdde99-f638-432a-9d9f-7e7aeccd1d41, process=sequential, number_of_agents=3, number_of_tasks=3)

### Kickoff the crew - let the magic happen

In [26]:
# Get your crew to work!
result = crew.kickoff()

 [2024-08-12 01:03:16][DEBUG]: == Working Agent: Senior Research Analyst
 [2024-08-12 01:03:16][INFO]: == Starting Task: Conduct a comprehensive research analysis to answer the question: Conduct investment analysis on Airbnb


> Entering new CrewAgentExecutor chain...
As a Senior Research Analyst, it is important to approach this task with a critical and analytical mindset. I must consider all factors and potential outcomes in order to provide a comprehensive research analysis on conducting investment analysis on Airbnb.

Action: duckduckgo_search
Action Input: {'query': 'Investment analysis on Airbnb'} 

Back To Blogs. Airbnb Investment Guide: Strategies & Opportunities. March 11, 2024. Copy link. ARTICLE SUMMARY. Unlock the secrets to successful Airbnb investing with strategic property selection, savvy data analysis, and risk mitigation to boost your profits in the competitive short-term rental market. QUICK NAVIGATION. The average cost to invest in an Airbnb rental business is betwe

In [27]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

My best complete final answer to the task is a detailed research report that addresses all aspects of investing in an Airbnb rental business. This report not only covers the financial aspect, but also takes into consideration other important factors such as location, competition, and target market.

To begin, it is important to understand the average cost of investing in an Airbnb rental business. While the report mentions this cost, it lacks specificity in terms of how it is calculated. Therefore, I have included a breakdown of the different expenses involved, such as rental price, property insurance, and monthly utilities, and how they contribute to the overall cost. This provides a more accurate and detailed understanding of the financial aspect of investing in an Airbnb rental business.

However, a successful investment in Airbnb goes beyond just the financial aspect. Location, competition, and target market are crucial factors that can greatly impact the success of an investment. Therefore, my research analysis also delves into these areas, providing insights on how to choose the right location, assess competition, and target the right market for an Airbnb rental business.

Furthermore, to support the analysis, I have included

# Heirarchical

In [29]:
import os
from crewai import Agent, Task, Crew, Process

# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal=f'Conduct comprehensive research to answer the question {question_to_answer}. With a particular focus on thinking about second order effects.',
  backstory="""You work at a leading analysis firm.
  Your expertise lies in analyzing complex issues.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        search_tool
      ]
)

critic = Agent(
  role='Critic',
  goal='Criticize output of research analysts to ensure the content is really insightful and not generic',
  backstory="""You are a senior analyst with an IQ of 125. You care alot about research and want to ensure insights are actionable""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)

writer = Agent(
  role='Research Writer',
  goal=f'Craft a compelling research report to answer the question: {question_to_answer}',
  backstory="""You are a renowned Research Strategist, known for
  your insightful and engaging research.
  You transform complex concepts into compelling reports. Include a section on potential second order effects.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)

# Define your task
task = Task(
    description=f"Conduct a comprehensive research analysis to answer the question: {question_to_answer}.",
    expected_output="A short concise report with insights for an investment analyst",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
)

# Instantiate your crew with a custom manager
crew = Crew(
    agents=[researcher, writer, critic],
    tasks=[task],
    manager_agent=manager,
    process=Process.hierarchical,
)

# Start the crew's work
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
As a Senior Research Analyst, it is crucial to conduct comprehensive research and analysis in order to provide accurate and actionable insights. In this case, it is important to not only look at Airbnb's current financial health, but also consider potential second order effects that may impact its future performance.

Action: duckduckgo_search
Action Input: {'query': 'Airbnb financial analysis'} 

Q4 and full-year 2023 financial results. Here is a snapshot of our Q4 and Full-Year 2023 results: Q4 revenue was $2.2 billion, up 17% year-over-year. Revenue increased to $2.2 billion in Q4 2023 from $1.9 billion in Q4 2022, driven by solid growth in Nights and Experiences Booked, a modest increase in Average Daily Rate ("ADR") and an FX ... Analysis of Airbnb's operational strengths and potential risks. ... Financial Health: Airbnb's financial health is a testament to its operational efficiency and strategic foresight. The company's ... Airbnb, Inc

In [30]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

### Investment Analysis Report on Airbnb

#### Financial Health:
Airbnb has demonstrated robust financial performance in recent years. Specifically, the company reported a 17% increase in revenue in Q4 2023 compared to the previous year, showcasing strong revenue growth. Airbnb has maintained operational efficiency and strategic foresight, contributing to its profitability and financial health. The company also has a positive cash flow and a healthy balance sheet with minimal debt, which is crucial for its long-term sustainability.

#### Market Position:
Airbnb is a dominant force in the global hospitality industry, offering a unique and innovative platform for short-term rentals. Since its inception in 2008, Airbnb has rapidly expanded its presence to over 220 countries and regions. The company boasts a strong market position with a valuation of $31 billion in 2017 and projected revenue of $8.5 billion in 2020. Airbnb's brand reputation and loyal customer base of over 150 million users worldwide further solidify its market position.

#### Competitive Landscape:
The short-term rental market is highly competitive, with Airbnb facing competition from traditional hotel chains and other online rental platforms, such as Booking.com and Vrbo. Despite this competition, Airbnb has a unique advantage through its focus on the sharing economy, offering a wide range of accommodations at competitive prices. This differentiation has allowed Airbnb to carve out a significant niche in the market.

#### Regulatory Environment:
Airbnb operates in a complex regulatory environment, with many cities and countries imposing strict regulations on short-term rentals. These regulations can impact Airbnb's growth and profitability, as compliance may require adjustments to its business model. The company has faced challenges in various locations due to these regulatory issues and must navigate this landscape carefully to sustain its operations.

#### Potential Risks and Challenges:
1. **Regulatory Risks:** The most significant risk for Airbnb comes from the regulatory landscape. Strict regulations on short-term rentals in various cities and countries could hinder the company's growth and profitability.
2. **Negative Publicity:** Airbnb has faced criticism for contributing to gentrification and driving up housing prices, leading to potential backlash from local communities and governments.
3. **Trust and Safety:** The business model's reliance on user trust poses a risk. Issues such as fraudulent listings, scams, and safety concerns could compromise user trust and affect Airbnb's reputation and financial performance.

#### Ethical Considerations:
Airbnb's impact on local communities and the housing market raises several ethical questions. The company has been accused of driving up rental prices and displacing long-term residents, particularly in popular tourist destinations. These issues highlight the need for Airbnb to balance its growth with its responsibility to the communities it operates in.

### Conclusion
Airbnb presents a unique and innovative investment opportunity with strong financial health, a dominant market position, and a competitive edge in the sharing economy. However, potential investors must carefully consider the regulatory risks, public perception challenges, and ethical implications associated with the company's business model. Airbnb's ability to adapt and navigate these challenges will be crucial for its sustained growth and success in the dynamic marketplace.